In [2]:
# Hack to make the module importable
import sys
sys.path.append(r'./../../')

In [10]:
%load_ext autoreload
%autoreload 2
from py2neo import Graph
import requests
import pyodata

from rel2graph.relational_modules.odata import ODataListIterator
from rel2graph import Converter, register_attribute_postprocessor, Attribute
import rel2graph.common_modules

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# Configure Logging
import logging

logging.basicConfig(level=logging.WARNING)
logging.getLogger("rel2graph").setLevel(logging.DEBUG)

In [13]:
graph = Graph(scheme="http", host="localhost", port=7474,  auth=('neo4j', 'password'))

graph.delete_all()  

In [14]:
###### CONFIGURATIONS #########
SERVICE_URL = 'https://ws.parlament.ch/odata.svc/'
HTTP_LIB = requests.Session()
client = pyodata.Client(SERVICE_URL, HTTP_LIB)

# Initialize node and relation dumps
nodes = []
relations = []

target = client.entity_sets.Person.get_entity(ID=4049,Language="DE").execute()

INFO:pyodata.client:Fetching metadata
INFO:pyodata.client:Creating OData Schema (version: 2)
INFO:pyodata.client:Creating OData Service (version: 2)
INFO:pyodata.service:Getting entity Person for key None and args {'ID': 4049, 'Language': 'DE'}


In [15]:
iterator = ODataListIterator([target])

In [16]:
schema_config = "./odata_demo_config.yaml"
converter = Converter(schema_config, iterator, graph, num_workers=5)

DEBUG:rel2graph.core.config_parser:Parsing config file './odata_demo_config.yaml'
DEBUG:rel2graph.core.config_parser:Compiling ENTITY("Person") : {'NODE("Person") person': ['ID = Person.ID', 'FirstName = Person.FirstName', 'LastName = append(Person.LastName, " I appended something")', 'Astaticattr = "HelloWorld!"'], 'CONDITION(NODE("Language"), "resource[\'FirstName\'] == \'Matthias\'") language': ['ShortName = ASSERT_EQ(Person.NativeLanguage, "D")'], 'RELATION(person, "speaks", language)': ['Since = Person.DateOfBirth']}
DEBUG:rel2graph.core.config_parser:Compiling module 'AttributeFactory' with static args ['ID', 'ID']
DEBUG:rel2graph.core.factories.registrar:Requested factory 'AttributeFactory' from registry.
DEBUG:rel2graph.core.config_parser:Compiling module 'AttributeFactory' with static args ['FirstName', 'FirstName']
DEBUG:rel2graph.core.factories.registrar:Requested factory 'AttributeFactory' from registry.
DEBUG:rel2graph.core.config_parser:Compiling module 'AttributeFactory'

In [17]:
converter()

INFO:rel2graph.core.converter:Starting creation of nodes.
DEBUG:rel2graph.core.converter:Starting Worker 0
DEBUG:rel2graph.core.converter:Starting Worker 1
DEBUG:rel2graph.core.converter:Starting Worker 2
DEBUG:rel2graph.core.converter:Starting Worker 3
DEBUG:rel2graph.core.converter:Worker 0: Processing resource type: 'Person'
DEBUG:rel2graph.core.converter:Starting Worker 4
DEBUG:rel2graph.core.converter:Exiting Worker 1
DEBUG:rel2graph.core.converter:Exiting Worker 2
DEBUG:rel2graph.core.converter:Exiting Worker 3
DEBUG:rel2graph.core.converter:Exiting Worker 4
DEBUG:rel2graph.core.converter:Exiting Worker 0
INFO:rel2graph.core.converter:Starting creation of relations.
DEBUG:rel2graph.core.converter:Starting Worker 0
DEBUG:rel2graph.core.converter:Starting Worker 1
DEBUG:rel2graph.core.converter:Starting Worker 2
DEBUG:rel2graph.core.converter:Starting Worker 3
DEBUG:rel2graph.core.converter:Worker 0: Processing resource type: 'Person'
DEBUG:rel2graph.core.converter:Starting Worker 

In [18]:
@register_attribute_postprocessor
def append(attribute, string):
    new_attr = Attribute(attribute.key, attribute.value + string)
    return new_attr

DEBUG:rel2graph.core.factories.registrar:Registered attribute postprocessor 'append''.
